In [1]:
!pip install bitsandbytes==0.40.2 peft trl==0.4.7 mlflow accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.6/147.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [2]:
!git clone https://github.com/harsh19/Shakespearizing-Modern-English.git data

Cloning into 'data'...
remote: Enumerating objects: 140, done.
remote: Total 140 (delta 0), reused 0 (delta 0), pack-reused 140
Receiving objects: 100% (140/140), 247.18 MiB | 14.51 MiB/s, done.
Resolving deltas: 100% (42/42), done.
Updating files: 100% (38/38), done.


In [3]:
import requests
import pandas as pd
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import pickle
import random
import numpy as np
import torch
from torch.utils.data import Dataset
import pandas as pd
from peft import LoraConfig
from transformers import TrainingArguments
from datasets import load_dataset
import json
from datasets import Dataset
import mlflow
from trl import SFTTrainer
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
import re
from tqdm import tqdm

In [4]:
r = 4
alpha = 0.05
train_name = 'lora_finetune_r_{}'.format(r)

In [5]:
def load_file(path):
  data = []
  data_file = open("data/data/" + path)
  for line in data_file:
    data.append(line.split('\n')[0])
  return data

In [6]:
def load_dataset(mode):
  template = """{} Or Shakespeare would write """
  train_modern = load_file('{}.{}.nltktok'.format(mode, 'modern'))
  train_original = load_file('{}.{}.nltktok'.format(mode, 'original'))
  train_df = pd.DataFrame({'prompt':train_modern, 'response':train_original})
  train_df['prompt'] = train_df["prompt"].apply(lambda x: template.format(x))
  train_df['response'] =  train_df['response']
  train_df = train_df[['prompt', 'response']]
  if( mode == 'train') or (mode == 'valid'):
    train_df['text'] = train_df["prompt"] + train_df["response"]
    train_df.drop(columns=['prompt', 'response'], inplace=True)
  return train_df

In [7]:
train_dataset = load_dataset('train')
validation_dataset = load_dataset('valid')

In [8]:
print("==> Train Dataset Example:\n", train_dataset.iloc[0]['text'])
print()
print("==> Validation Dataset Example:\n", validation_dataset.iloc[0]['text'])

==> Train Dataset Example:
 I have half a mind to hit you before you speak again . Or Shakespeare would write I have a mind to strike thee ere thou speak'st .

==> Validation Dataset Example:
 Now , you lie there on the path . Or Shakespeare would write Lie thou there ( throwing down a letter ) , for here comes the trout that must be caught with tickling .


In [9]:
train_dataset = Dataset.from_pandas(train_dataset)
validation_dataset = Dataset.from_pandas(validation_dataset)

In [10]:
checkpoint = "bigscience/mt0-base"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/430 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

In [11]:
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint, torch_dtype="auto", device_map="auto")

config.json:   0%|          | 0.00/798 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

In [12]:
inputs = tokenizer.encode("Translate to English: Je t’aime.", return_tensors="pt").to("cuda")
outputs = model.generate(inputs)
print(tokenizer.decode(outputs[0]))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


<pad> I love you.</s>


## LoRA Config

In [13]:
model_modules = str(model.modules)
pattern = r'\((\w+)\): Linear'
linear_layer_names = re.findall(pattern, model_modules)
target_modules = list(set(linear_layer_names))

In [14]:
target_modules

['lm_head', 'k', 'v', 'wi_0', 'wi_1', 'q', 'wo', 'o']

In [15]:
lora_config = LoraConfig(
    r=r,
    lora_alpha=alpha,
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_2_SEQ_LM",
)
adapted_model = get_peft_model(model, lora_config)

## Train Config

In [16]:
base_dir = "./"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = 'adamw_torch'
learning_rate = 5e-3
max_grad_norm = 1.0
warmup_ratio = 0.03
lr_scheduler_type = "linear"

training_args = TrainingArguments(
    output_dir=base_dir,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    num_train_epochs = 1.0,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    learning_rate=learning_rate,
    max_grad_norm=max_grad_norm,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

## Finetune Model

In [17]:
trainer = SFTTrainer(
    adapted_model,
    train_dataset=train_dataset,
    eval_dataset = validation_dataset,
    dataset_text_field="text",
    max_seq_length=256,
    args=training_args,
)

Map:   0%|          | 0/18395 [00:00<?, ? examples/s]

Map:   0%|          | 0/1218 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [18]:
with mlflow.start_run(run_name=train_name):
  trainer.train()

Epoch,Training Loss,Validation Loss
0,0.007400,0.000771


In [19]:
test_dataset = load_dataset('test')

In [20]:
test_dataset.head()

,prompt,response
0,A jumbled confession can only receive a jumble...,Riddling confession finds but riddling shrift .
1,I love rich Capulet's daughter . Or Shakespear...,Then plainly know my heart's dear love is set ...
2,We're bound to each other in every possible wa...,"As mine on hers , so hers is set on mine , And..."
3,I'll tell you more later about when and where ...,"When and where and how We met , we wooed and m..."
4,"Holy Saint Francis , this is a drastic change ...","Holy Saint Francis , what a change is here !"


In [21]:
output_data = []
for index, row in tqdm(test_dataset.iterrows()):
  prompt = row['prompt']
  expected_response = row['response']
  input_ids = tokenizer(prompt, return_tensors="pt").input_ids.cuda()
  generation_output = adapted_model.generate(input_ids=input_ids)
  model_response = tokenizer.decode(generation_output[0], skip_special_tokens=True)
  output_data.append([model_response, expected_response])

0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
1462it [12:23,  1.97it/s]


In [22]:
output_data = np.array(output_data)

In [23]:
test_df = pd.DataFrame({"model_response": output_data[:, 0], "expected_response":output_data[:, 1]})
test_df.to_pickle(f"./{train_name}.pkl")

In [24]:
with open(f"./{train_name}.pkl", "rb") as input_file:
  e = pickle.load(input_file)

In [25]:
e.head()

,model_response,expected_response
0,A jumbled confession can only receive a jumble...,Riddling confession finds but riddling shrift .
1,I love rich Capulet's daughter. Or Shakespeare...,Then plainly know my heart's dear love is set ...
2,We're bound to each other in every possible wa...,"As mine on hers , so hers is set on mine , And..."
3,I'll tell you more later about when and where ...,"When and where and how We met , we wooed and m..."
4,"Holy Saint Francis, this is a drastic change! ...","Holy Saint Francis , what a change is here !"


In [26]:
len(train_dataset)

18395